# FERNANDO LEON FRANCO

In [1]:
import threading
import time
import math
from concurrent.futures import ProcessPoolExecutor, ThreadPoolExecutor
import multiprocessing as mp
from queue import Queue
import random

# Ejercicio 1
Múltiples threads actualizan un contador compartido. Usa Lock para evitar las condiciones de carrera.

In [2]:
contador = 0
lock = threading.Lock()

def incrementar():
     global contador
     for _ in range(100000):
         with lock:
             contador += 1

def ejercicio_1():
    hilos = []
    for _ in range(4):
        hilo = threading.Thread(target=incrementar)
        hilos.append(hilo)
        hilo.start()

    for hilo in hilos:
        hilo.join()

    print(f"Contador final: {contador}")
ejercicio_1()

Contador final: 400000


# Ejercicio 2
Limitar a 3 respuestas simultáneos a una API externa usando semáforos.

In [3]:
semaforo = threading.Semaphore(3)

def llamar_api(id):
    with semaforo:
        print(f"API {id} accediendo al recurso")
        time.sleep(2)
    print(f"API {id} liberó el recurso")

def ejercicio_2():
    with ThreadPoolExecutor(max_workers=3) as executor:
        executor.map(llamar_api, range(20))

ejercicio_2()

API 0 accediendo al recurso
API 1 accediendo al recurso
API 2 accediendo al recurso
API 0 liberó el recursoAPI 1 liberó el recurso
API 3 accediendo al recurso
API 2 liberó el recurso
API 4 accediendo al recurso

API 5 accediendo al recurso
API 4 liberó el recurso
API 6 accediendo al recurso
API 3 liberó el recurso
API 7 accediendo al recurso
API 5 liberó el recurso
API 8 accediendo al recurso
API 6 liberó el recurso
API 9 accediendo al recurso
API 7 liberó el recurso
API 10 accediendo al recurso
API 8 liberó el recurso
API 11 accediendo al recurso
API 9 liberó el recurso
API 12 accediendo al recurso
API 10 liberó el recurso
API 13 accediendo al recurso
API 11 liberó el recurso
API 14 accediendo al recurso
API 12 liberó el recursoAPI 13 liberó el recurso
API 15 accediendo al recurso

API 16 accediendo al recurso
API 14 liberó el recurso
API 17 accediendo al recurso
API 15 liberó el recurso
API 18 accediendo al recurso
API 16 liberó el recurso
API 19 accediendo al recurso
API 17 liberó e

# Ejercicio 3
Un proceso principal debe esperar a que varios workers terminen su inicialización.

In [4]:
def worker(id, evento):
    print(f"Worker {id} inicializando...")
    time.sleep(2)
    print(f"Worker {id} listo")
    evento.set()

def ejercicio_3():
    num_workers = 3
    eventos = [threading.Event() for _ in range(num_workers)]

    for i in range(num_workers):
        threading.Thread(target=worker, args=(i, eventos[i])).start()

    print("Principal esperando que los workers terminen...")
    for ev in eventos:
        ev.wait()

    print("Todos los workers terminaron. Principal continúa.")

ejercicio_3()


Worker 0 inicializando...
Worker 1 inicializando...
Worker 2 inicializando...
Principal esperando que los workers terminen...
Worker 0 listo
Worker 1 listo
Worker 2 listo
Todos los workers terminaron. Principal continúa.


# Ejercicio 4
Múltiples modelos deben esperar a que todos completen una epoch antes de continuar.

In [5]:
barrera_4 = threading.Barrier(4)

def modelo(id, num_epochs):
    print(f"Modelo {id} inicializando...")
    time.sleep(2)
    barrera_4.wait()
    print(f"Modelo {id} empieza a trabajar")
    for i in range(num_epochs):
        print(f'Epoca {i} del modelo {id} terminada')
        barrera_4.wait()

def ejercicio_4():
    hilos = []
    for i in range(4):
        hilo = threading.Thread(target=modelo, args=(i,5))
        hilos.append(hilo)
        hilo.start()

    for hilo in hilos:
        hilo.join()
ejercicio_4()


Modelo 0 inicializando...
Modelo 1 inicializando...
Modelo 2 inicializando...
Modelo 3 inicializando...
Modelo 3 empieza a trabajar
Epoca 0 del modelo 3 terminada
Modelo 0 empieza a trabajar
Epoca 0 del modelo 0 terminada
Modelo 1 empieza a trabajar
Epoca 0 del modelo 1 terminada
Modelo 2 empieza a trabajar
Epoca 0 del modelo 2 terminada
Epoca 1 del modelo 3 terminada
Epoca 1 del modelo 0 terminada
Epoca 1 del modelo 1 terminada
Epoca 1 del modelo 2 terminada
Epoca 2 del modelo 3 terminada
Epoca 2 del modelo 0 terminada
Epoca 2 del modelo 1 terminada
Epoca 2 del modelo 2 terminada
Epoca 3 del modelo 3 terminada
Epoca 3 del modelo 0 terminada
Epoca 3 del modelo 2 terminada
Epoca 3 del modelo 1 terminada
Epoca 4 del modelo 0 terminada
Epoca 4 del modelo 1 terminada
Epoca 4 del modelo 3 terminada
Epoca 4 del modelo 2 terminada


# Ejercicio 5
Implementar un sistema productor-consumidor con buffer limitado.

In [6]:
buffer_size = 5
buffer = Queue(maxsize=buffer_size)

vacio = threading.Semaphore(buffer_size)
lleno = threading.Semaphore(0)

def productor(id):
    for i in range(5):
        item = f"Item-{id}-{i}"
        vacio.acquire()
        buffer.put(item)
        print(f"🟢 Productor {id} produjo {item}")
        lleno.release()
        time.sleep(random.random())

def consumidor(id):
    for i in range(5):
        lleno.acquire()
        item = buffer.get()
        print(f"🔴 Consumidor {id} consumió {item}")
        vacio.release()
        time.sleep(random.random())

def ejercicio_5():
    productores = [threading.Thread(target=productor, args=(i,)) for i in range(2)]
    consumidores = [threading.Thread(target=consumidor, args=(i,)) for i in range(2)]

    for t in productores + consumidores:
        t.start()
    for t in productores + consumidores:
        t.join()

ejercicio_5()


🟢 Productor 0 produjo Item-0-0
🟢 Productor 1 produjo Item-1-0
🔴 Consumidor 0 consumió Item-0-0
🔴 Consumidor 1 consumió Item-1-0
🟢 Productor 0 produjo Item-0-1
🔴 Consumidor 0 consumió Item-0-1
🟢 Productor 1 produjo Item-1-1
🔴 Consumidor 1 consumió Item-1-1
🟢 Productor 1 produjo Item-1-2
🔴 Consumidor 0 consumió Item-1-2
🟢 Productor 0 produjo Item-0-2
🔴 Consumidor 0 consumió Item-0-2
🟢 Productor 1 produjo Item-1-3
🔴 Consumidor 1 consumió Item-1-3
🟢 Productor 0 produjo Item-0-3
🟢 Productor 0 produjo Item-0-4
🔴 Consumidor 1 consumió Item-0-3
🔴 Consumidor 0 consumió Item-0-4
🟢 Productor 1 produjo Item-1-4
🔴 Consumidor 1 consumió Item-1-4


# Ejercicio 6
Múltiples lectores y un escritor acceden a una base de datos.

In [7]:
db = []
lock = threading.RLock()

def lector(id):
    with lock:
        print(f"Lector {id} leyendo: {db}")
        time.sleep(random.uniform(0.2,0.5))

def escritor(id, valor):
    with lock:
        print(f"Escritor {id} escribiendo {valor}")
        db.append(valor)
        time.sleep(random.uniform(0.2,0.5))

def ejercicio_6():
    hilos = []
    for i in range(3):  # 3 lectores
        t = threading.Thread(target=lector, args=(i,))
        hilos.append(t)
    for i in range(2):  # 2 escritores
        t = threading.Thread(target=escritor, args=(i,f"data-{i}"))
        hilos.append(t)

    for h in hilos: h.start()
    for h in hilos: h.join()

ejercicio_6()


👀 Lector 0 leyendo: []
👀 Lector 1 leyendo: []
👀 Lector 2 leyendo: []
✍️ Escritor 0 escribiendo data-0
✍️ Escritor 1 escribiendo data-1


# Ejercicio 7
Condition para notificaciones entre threads
Un thread espera hasta que cierta condición se cumpla.

In [8]:
condicion = threading.Condition()
dato_listo = False

def consumidor():
    global dato_listo
    with condicion:
        print("Consumidor esperando dato...")
        while not dato_listo:
            condicion.wait()
        print("Consumidor recibió notificación: dato listo")

def productor():
    global dato_listo
    time.sleep(2)
    with condicion:
        dato_listo = True
        print("Productor produjo el dato, notificando...")
        condicion.notify()

def ejercicio_7():
    t1 = threading.Thread(target=consumidor)
    t2 = threading.Thread(target=productor)
    t1.start(); t2.start()
    t1.join(); t2.join()

ejercicio_7()


🔴 Consumidor esperando dato...
🟢 Productor produjo el dato, notificando...
✅ Consumidor recibió notificación: dato listo


# Ejercicio 8
Procesar imágenes en paralelo con límite de GPU.

In [9]:
gpu_semaforo = threading.Semaphore(2)

def procesar_imagen(nombre):
    with gpu_semaforo:
        print(f"Procesando {nombre} en la GPU...")
        time.sleep(random.uniform(1, 3))
        print(f"{nombre} procesada")
    return f"{nombre} lista"

def ejercicio_8():
    imagenes = [f"imagen_{i}.jpg" for i in range(10)]
    resultados = []
    with ThreadPoolExecutor(max_workers=5) as executor:
        futuros = [executor.submit(procesar_imagen, img) for img in imagenes]
        for futuro in futuros:
            resultados.append(futuro.result())
    print("Resultados finales:", resultados)

ejercicio_8()

Procesando imagen_0.jpg en la GPU...Procesando imagen_1.jpg en la GPU...

imagen_0.jpg procesada
Procesando imagen_5.jpg en la GPU...
imagen_1.jpg procesada
Procesando imagen_6.jpg en la GPU...
imagen_6.jpg procesada
Procesando imagen_7.jpg en la GPU...
imagen_5.jpg procesada
Procesando imagen_8.jpg en la GPU...
imagen_7.jpg procesada
Procesando imagen_9.jpg en la GPU...
imagen_8.jpg procesada
Procesando imagen_4.jpg en la GPU...
imagen_4.jpg procesada
Procesando imagen_2.jpg en la GPU...
imagen_9.jpg procesada
Procesando imagen_3.jpg en la GPU...
imagen_2.jpg procesada
imagen_3.jpg procesada
Resultados finales: ['imagen_0.jpg lista', 'imagen_1.jpg lista', 'imagen_2.jpg lista', 'imagen_3.jpg lista', 'imagen_4.jpg lista', 'imagen_5.jpg lista', 'imagen_6.jpg lista', 'imagen_7.jpg lista', 'imagen_8.jpg lista', 'imagen_9.jpg lista']


# Ejercicio 9
Procesamiento CPU-intensivo con límite de procesos.

In [10]:
import multiprocessing as mp
import time
import random

def init_globals(s):
    global semaforo
    semaforo = s  # cada proceso recibe el mismo semáforo compartido

def tarea_intensiva(n):
    """Simula una tarea CPU-intensiva sin usar factorial"""
    semaforo.acquire()  # Adquirir el semáforo
    try:
        print(f"🔵 {mp.current_process().name} iniciando tarea {n}...")

        # Simulación de trabajo CPU-intensivo
        inicio = time.time()
        # Operación matemática intensiva (sumar muchos números)
        resultado = 0
        for i in range(n * 1000000):
            resultado += i * 0.000001

        tiempo_ejecucion = time.time() - inicio
        time.sleep(random.uniform(0.1, 0.3))  # Pequeña pausa adicional

        print(f"✅ Tarea {n} completada en {tiempo_ejecucion:.2f}s")
        return resultado

    finally:
        semaforo.release()  # Liberar el semáforo

def ejercicio_9():
    """Multiprocessing con Pool y semáforos - Procesamiento CPU-intensivo con límite"""
    # Datos de entrada para procesar
    tareas = [10, 15, 20, 25, 30, 35]  # Números que definen la intensidad

    print("🚀 Iniciando multiprocessing con semáforos...")
    print(f"📊 Tareas a procesar: {tareas}")
    print(f"🎯 Límite de procesos simultáneos: 3")
    print("-" * 50)

    # Crear Manager para el semáforo compartido
    with mp.Manager() as manager:
        # Semáforo que permite máximo 3 procesos simultáneos
        semaforo_compartido = manager.Semaphore(3)

        # Crear Pool de procesos con inicialización del semáforo
        with mp.Pool(
            processes=6,  # 6 procesos en total
            initializer=init_globals,
            initargs=(semaforo_compartido,)
        ) as pool:

            # Ejecutar las tareas en paralelo con límite de concurrencia
            resultados = pool.map(tarea_intensiva, tareas)

    print("-" * 50)
    print("📦 Resultados finales:")
    for i, (tarea, resultado) in enumerate(zip(tareas, resultados)):
        print(f"  Tarea {tarea}: {resultado:.2f}")

if __name__ == "__main__":
    # Esto es importante para Windows
    mp.freeze_support()
    ejercicio_9()

🚀 Iniciando multiprocessing con semáforos...
📊 Tareas a procesar: [10, 15, 20, 25, 30, 35]
🎯 Límite de procesos simultáneos: 3
--------------------------------------------------
🔵 ForkPoolWorker-7 iniciando tarea 25...🔵 ForkPoolWorker-2 iniciando tarea 10...🔵 ForkPoolWorker-3 iniciando tarea 15...


✅ Tarea 10 completada en 2.16s
🔵 ForkPoolWorker-4 iniciando tarea 20...
✅ Tarea 15 completada en 3.11s
🔵 ForkPoolWorker-6 iniciando tarea 30...
✅ Tarea 25 completada en 5.40s
🔵 ForkPoolWorker-5 iniciando tarea 35...
✅ Tarea 20 completada en 3.99s
✅ Tarea 30 completada en 5.70s
✅ Tarea 35 completada en 5.45s
--------------------------------------------------
📦 Resultados finales:
  Tarea 10: 49999995.00
  Tarea 15: 112499992.50
  Tarea 20: 199999990.00
  Tarea 25: 312499987.50
  Tarea 30: 449999985.00
  Tarea 35: 612499982.50


# Ejercicio 10
Sistema completo de entrenamiento distribuido
Coordinar múltiples componentes de un sistema de IA.

In [15]:
buffer = Queue()

# Barrera para sincronizar workers por epoch
num_workers = 3
barrera = threading.Barrier(num_workers)

# Evento para notificar al monitor
evento_epoch = threading.Event()

def productor():
    for batch in range(20):
        buffer.put(f"batch-{batch}")
        print(f"📦 Productor generó batch-{batch}")
        time.sleep(0.1)

def worker(id, num_epochs=3):
    for epoch in range(num_epochs):
        batch = buffer.get()
        print(f"🔵 Worker {id} entrenando con {batch} en epoch {epoch}")
        time.sleep(random.uniform(0.2,0.5))  # simula forward+backward
        buffer.task_done()
        barrera.wait()  # esperar a que todos terminen epoch
        if id == 0:  # solo un worker dispara evento al monitor
            evento_epoch.set()

def monitor(num_epochs=3):
    for epoch in range(num_epochs):
        evento_epoch.wait()
        print(f"📊 Monitor: epoch {epoch} completada")
        evento_epoch.clear()

def ejercicio_10():
    hilos = []
    # productor
    t_prod = threading.Thread(target=productor)
    t_prod.start()
    hilos.append(t_prod)

    # monitor
    t_mon = threading.Thread(target=monitor)
    t_mon.start()
    hilos.append(t_mon)

    # workers
    for i in range(num_workers):
        t = threading.Thread(target=worker, args=(i,))
        t.start()
        hilos.append(t)

    for h in hilos:
        h.join()

ejercicio_10()

📦 Productor generó batch-0
🔵 Worker 0 entrenando con batch-0 en epoch 0
📦 Productor generó batch-1
🔵 Worker 1 entrenando con batch-1 en epoch 0
📦 Productor generó batch-2
🔵 Worker 2 entrenando con batch-2 en epoch 0
📦 Productor generó batch-3
📦 Productor generó batch-4
📦 Productor generó batch-5
📦 Productor generó batch-6
🔵 Worker 2 entrenando con batch-3 en epoch 1
🔵 Worker 1 entrenando con batch-4 en epoch 1
🔵 Worker 0 entrenando con batch-5 en epoch 1
📊 Monitor: epoch 0 completada
📦 Productor generó batch-7
📦 Productor generó batch-8
📦 Productor generó batch-9
📦 Productor generó batch-10
🔵 Worker 0 entrenando con batch-6 en epoch 2
🔵 Worker 1 entrenando con batch-7 en epoch 2
🔵 Worker 2 entrenando con batch-8 en epoch 2
📊 Monitor: epoch 1 completada
📦 Productor generó batch-11
📦 Productor generó batch-12
📦 Productor generó batch-13
📦 Productor generó batch-14
📦 Productor generó batch-15
📊 Monitor: epoch 2 completada
📦 Productor generó batch-16
📦 Productor generó batch-17
📦 Productor